In [1]:
from utils import uniprotRetrieve
import pandas as pd
import subprocess
from tqdm import tqdm
import tensorflow as tf #Make sure version 1.3.0 is installed
import numpy as np
import os

def cdhit(inputFastaFile, identity=0.9, outputDir=""):
    if outputDir and not os.path.exists(outputDir):
        os.makedirs(outputDir)
    outputFile=inputFastaFile.split("/")[-1].replace(".fasta","_cdhit.fasta")
    outputFilePath="{}{}".format(outputDir,outputFile)
    command = "cd-hit -i {} -o {} -c {}  -n 3 -M 1500".format(inputFastaFile,outputFilePath,identity)
    cmd = subprocess.Popen(command,
                           shell=True,
                           stdin=subprocess.PIPE,
                          stdout=subprocess.PIPE,
                          stderr=subprocess.STDOUT)
    cmd.communicate()
    return outputFilePath

""" Read in model to generate Unirep vectors"""
# Sync relevant weight files
!aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
# Import the mLSTM babbler model
from unirep import babbler64 as babbler

# Where model weights are stored.
MODEL_WEIGHT_PATH = "./64_weights"

batch_size = 12
b = babbler(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH)

/home/jan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:461: FutureWarnin

# Biodata Mining

To get the dataset,
the **UniProt REST API** was used to gather cytoplasmic and periplasmic proteins.
To limit the sampling bias, the software **CD-HIT** compressed the proteins sequence information so that only clusters of at most 50 percent sequence identity were left.
Finally the sequences were transformed to a fixed length vector representation using **UniRep**.
This methods extracts states from an unsupervised trained mLSTM-RNN and combines them into a fixed length UniRep representation.
This representation contains essential structural and functional features that can be used by ML algorithms to distinguish between Periplasmic and Cytoplasmic proteins.

Traditionally, 
the detection of a signal peptide is used to make this distinction.
However it has been observed that the signal peptide itself is insufficient to guarantee periplasmic exportation.
Therefore,
a dataset was made with unirep vectors using the full periplasmic protein sequence 
and another one using the sequence with the signal peptide cut off. 

# 1. Download data

### 1.1 Cytoplasmic proteins

To obtain cytoplasmic proteins,
the [Uniprot REST api](https://www.uniprot.org/help/api%5Fqueries) was used to perform queries.
To limit the dataset a bit, 
we only included proteins of the **Gammaproteobacteria** taxum.
In addition we asked the query only those proteins with an **annotation of being in the Cytoplasm of cytosol**.
Finally, as an extra safeguard,
proteins **could not have an annotation of containing a signal peptide**.

In [2]:
"""Download sequences and identifiers"""
QUERY="taxonomy:Gammaproteobacteria (locations:(location:cytoplasm) OR locations:(location:cytosol)) NOT annotation:(type:signal)"
FORMAT="tab"
FILENAME="cytoplasm.tab"
COLUMNS="id,sequence"
# Use uniprot REST API to retrieve protein in tab format
uniprotRetrieve(FILENAME, format=FORMAT, query=QUERY, columns=COLUMNS)


CYTOPLASM = pd.read_csv(FILENAME,sep="\t")
# Write fasta file 
fastaFile = open("cytoplasm.fasta","w")
# Transform tab format to fasta format
for i,row in tqdm(CYTOPLASM.iterrows()):
    entry = row["Entry"]
    sequence = row["Sequence"]
    fastaFile.write(">{}\n{}\n".format(entry,sequence))
fastaFile.close()

1970892it [08:32, 3846.91it/s]


The software **CD-HIT** aligns the different protein sequences in the dataset and calculates the **sequence identity** percentage.
Similar clusters were combined into clusters and representatives were chosen 
so that each representative has at most 50 percent sequence identity with any other representative.

In [ ]:
""" Use cdhit to obtain dataset of no more than 50 percent sequences identit """
CYTOPLASM_CDHIT = cdhit("cytoplasm.fasta",identity=0.5)

### 1.1.2 Periplasmic proteins

A similar method was used to obtain the Periplasmic protein sequences.
This time, 
the query asked for **annotations of the protein being in the periplasm**,
and **having a signal peptide**.
The proteins sequences and there signal peptide locations are downloaded into a tab-separated text file.
From here,
two fasta files were generated:
* one including the signal peptide sequence 
* one excluding the signal peptide sequence

In [ ]:
QUERY="taxonomy:Gammaproteobacteria locations:(location:periplasm) annotation:(type:signal)"
FORMAT="tab"
FILENAME="periplasm.tab"
COLUMNS="id,sequence,feature(SIGNAL)"
uniprotRetrieve(FILENAME, format=FORMAT, query=QUERY, columns=COLUMNS)

PERIPLASM = pd.read_csv(FILENAME,sep="\t")
PERIPLASM["newStart"]=PERIPLASM["Signal peptide"].apply(lambda x : int(x.split(";")[0].split("..")[-1]))
# Write fasta file with and without fasta 
withSP = open("periplasm_with_sp.fasta","w")
withoutSP = open("periplasm_without_sp.fasta","w")
for i,row in tqdm(PERIPLASM.iterrows()):
    entry = row["Entry"]
    sequence = row["Sequence"]
    newStart = row["newStart"]
    withSP.write(">{}\n{}\n".format(entry,sequence))
    withoutSP.write(">{}\n{}\n".format(entry,sequence[newStart:]))
withSP.close()
withoutSP.close()

Again, **CD-HIT** was used to reduce sampling bias.
Threshold was set on **50 percent sequence identity**.

In [ ]:
""" Use cdhit to obtain dataset of no more than 50 percent sequences identity (without SP) """
PERIPLASM_WITHOUT_SP_CDHIT = cdhit("periplasm_without_sp.fasta",identity=0.5)

In [ ]:
""" Use cdhit to obtain dataset of no more than 50 percent sequences identity (with SP) """
PERIPLASM_WITH_SP_CDHIT = cdhit("periplasm_with_sp.fasta",identity=0.5)

### 1.1.3 Generate unirep vector

The **UniRep** method was used to generate fixed length vectors.
(a more elaborate explanation should come here that stresses the power of unsupervised feature extraction).

In [ ]:
""" Generate vectors for cytoplasmic proteins """

with open("cytoplasm.fasta") as f:
    lines = [line.strip() for line in f.readlines()]

cytoplasm=dict()
for line in lines:
    if line.startswith(">"):
        ID = line.replace(">","")
        cytoplasm[ID]=dict()
        cytoplasm[ID]["seq"]=""
    else:
        cytoplasm[ID]["seq"]+=line

FILE="cytoplasm.unirep"
if not os.path.exists(FILE):
    with open(FILE, 'w'): pass      

with open(FILE) as f:
    IDS_DONE = [LINE.strip().replace(">","") for LINE in f.readlines() if LINE.startswith(">")]
IDS_TODO = [ID for ID in cytoplasm.keys() if ID not in IDS_DONE]

for ID in tqdm(IDS_TODO):
    seq = cytoplasm[ID]["seq"]
    repres = b.get_rep(seq)
    #cytoplasm[ID]["unirep"]= repres
    with open(FILE,"ab") as f:
        f.write(bytes(">"+ID+"\n", 'utf-8'))
        np.savetxt(f,repres)

In [ ]:
""" Generate vectors for periplamic proteins without SP """

with open("periplasm_without_sp_cdhit.fasta") as f:
    lines = [line.strip() for line in f.readlines()]

periplasm=dict()
for line in lines:
    if line.startswith(">"):
        ID = line.replace(">","")
        periplasm[ID]=dict()
        periplasm[ID]["seq"]=""
    else:
        periplasm[ID]["seq"]+=line

FILE="periplasm_without_sp.unirep"
if not os.path.exists(FILE):
    with open(FILE, 'w'): pass      

with open(FILE) as f:
    IDS_DONE = [LINE.strip().replace(">","") for LINE in f.readlines() if LINE.startswith(">")]
IDS_TODO = [ID for ID in periplasm.keys() if ID not in IDS_DONE]

for ID in tqdm(IDS_TODO):
    seq = periplasm[ID]["seq"]
    repres = b.get_rep(seq)
    #periplasm[ID]["unirep"]= repres
    with open(FILE,"ab") as f:
        f.write(bytes(">"+ID+"\n", 'utf-8'))
        np.savetxt(f,repres)

In [ ]:
""" Generate vectors for periplamic proteins with SP """

with open("periplasm_with_sp_cdhit.fasta") as f:
    lines = [line.strip() for line in f.readlines()]

periplasm=dict()
for line in lines:
    if line.startswith(">"):
        ID = line.replace(">","")
        periplasm[ID]=dict()
        periplasm[ID]["seq"]=""
    else:
        periplasm[ID]["seq"]+=line

FILE="periplasm_with_sp.unirep"
if not os.path.exists(FILE):
    with open(FILE, 'w'): pass      

with open(FILE) as f:
    IDS_DONE = [LINE.strip().replace(">","") for LINE in f.readlines() if LINE.startswith(">")]
IDS_TODO = [ID for ID in periplasm.keys() if ID not in IDS_DONE]

for ID in tqdm(IDS_TODO):
    seq = periplasm[ID]["seq"]
    repres = b.get_rep(seq)
    #periplasm[ID]["unirep"]= repres
    with open(FILE,"ab") as f:
        f.write(bytes(">"+ID+"\n", 'utf-8'))
        np.savetxt(f,repres)